In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import wandb

import numpy as np
import seaborn as sns
import torch

import sys
BASE_PATH = globals()['_dh'][0].parent.absolute()
sys.path.insert(1, str(BASE_PATH))

from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from src.utils.utils import make_Planetoid_data, exp_per_model, make_uniform_schedule
from src.utils.metrics import MAD
from src.models.iterativeModels import learnable_adaptive_iGCN, iterativeGCN
from src.models.GCN import GCN
import wandb
wandb.login()

/Users/sizhuang/Desktop/GitHubRepos/IterativeMethods/iterENV/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/Users/sizhuang/Desktop/GitHubRepos/IterativeMethods/iterENV/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
wandb: Currently logged in as: sizhuang. Use `wandb login --relogin` to force relogin


True

In [2]:
hyper = {
    'num_epochs': 200,
    'dataset_name': "Cora",
    'noise_percent': 0,
    'hid_dim': 32,
    'num_iter_layers': 2,
    'smooth_fac': 0.6,
    'dropout': 0.5,
    'learning_rate': 0.005,
    'weight_decay': 4e-4
} 
data, num_features, num_classes = make_Planetoid_data(hyper)

In [3]:
wandb.init(config=hyper, 
               job_type="run_iGCN", 
               project="IterativeMethods", 
               tags=["iterativeGCN"])
config = wandb.config
train_schedule = make_uniform_schedule(config.num_iter_layers, config.smooth_fac)
data, num_features, num_classes = make_Planetoid_data(config)
wandb.log({
    'train_schedule': train_schedule
})

model = iterativeGCN(input_dim=num_features,
                                output_dim=num_classes,
                                hidden_dim=config.hid_dim,
                                train_schedule=train_schedule,
                                dropout=config.dropout)
exp_per_model(model, data, config)
wandb.finish()

Final MAD: 
0.0004354462777726567
Final MAD: 
3.613320931656624e-05
Final MAD: 
0.0005434565902841745
Final MAD: 
6.879717228893706e-05
Final MAD: 
0.0010165288761735324
Final MAD: 
0.00012666166665349705
Final MAD: 
0.0016365115863868323
Final MAD: 
0.00022177992106865306
Final MAD: 
0.0027064798987974136
Final MAD: 
0.0003815539215552296
Final MAD: 
0.004397788152042048
Final MAD: 
0.0006196237641213981
Final MAD: 
0.006328905538225549
Final MAD: 
0.0009939422536487388
Final MAD: 
0.010319611727824224
Final MAD: 
0.0015738173884780316
Final MAD: 
0.01474892585678943
Final MAD: 
0.0024540455258448704
Final MAD: 
0.021899565419472543
Final MAD: 
0.0038528906686234486
Final MAD: 
0.032653334504095424
Final MAD: 
0.006068800205549221
Final MAD: 
0.04624239928264935
Final MAD: 
0.009546651504560012
Final MAD: 
0.07058068386220906
Final MAD: 
0.015077980260247613
Final MAD: 
0.10077638617446785
Final MAD: 
0.02352394672934005
Final MAD: 
0.146846299363195
Final MAD: 
0.036190793706387905
F

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
num_param,▁
test_accuracy,▁
test_loss,▁
training_accuracy,▁▂▂▂▄▄▄▄▆▅▅▅▅▆▅▆▆▅▇▆▇▇▇▇▇█▇███▇▇▇███████
training_loss,██████▇▇▇▇▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▄▄▄▅▅▄▄▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇███▇████▇██████
validation_loss,███████▇▇▇▆▆▅▅▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▂▂▁▁
epoch,200
num_param,23335
test_accuracy,0.71


In [4]:
out = model(data.x, data.edge_index)
torch.std(out, dim=0)

Final MAD: 
0.8523545114110451


tensor([2.5018, 2.9722, 2.4346, 3.2920, 3.0870, 2.5078, 3.6637],
       grad_fn=<StdBackward0>)

In [6]:
wandb.init(job_type="run_GCN", 
               project="IterativeMethods", 
               config=hyper, 
               notes="usualGCN",
               tags=["usualGCN"])
config = wandb.config
model2 = GCN(input_dim=num_features,
                                output_dim=num_classes,
                                hidden_dim=config.hid_dim,
                                num_layers=config.num_iter_layers,
                                dropout=config.dropout,
                                )
exp_per_model(model2, data, config)

Final MAD: 
0.5764086983153955
Final MAD: 
0.19553517047439148
Final MAD: 
0.3026360339249142
Final MAD: 
0.5588939791527745
Final MAD: 
0.7062593338886248
Final MAD: 
0.6684230702501878
Final MAD: 
0.7935475660426731
Final MAD: 
0.5144803299000305
Final MAD: 
0.7003733893562414
Final MAD: 
0.4742005734364055
Final MAD: 
0.6961104826884376
Final MAD: 
0.47571138106398053
Final MAD: 
0.7283901908778946
Final MAD: 
0.5060473615490493
Final MAD: 
0.714991542830761
Final MAD: 
0.5033808020880464
Final MAD: 
0.7163055842791848
Final MAD: 
0.53429237595534
Final MAD: 
0.712499119926162
Final MAD: 
0.5829526063934303
Final MAD: 
0.7663094034203021
Final MAD: 
0.6266852940839545
Final MAD: 
0.7932416151106915
Final MAD: 
0.6710777013878578
Final MAD: 
0.8233473261377543
Final MAD: 
0.7054332724411835
Final MAD: 
0.8282268549988929
Final MAD: 
0.7276987995657537
Final MAD: 
0.8566284281502051
Final MAD: 
0.7279606293797137
Final MAD: 
0.8508244372772061
Final MAD: 
0.7077413229837907
Final MAD:

In [7]:
out2 = model2(data.x, data.edge_index)
torch.std(out2, dim=0)

Final MAD: 
0.8092150742072397


tensor([0.9069, 0.8196, 0.9754, 1.1887, 1.0602, 1.0512, 1.0446],
       grad_fn=<StdBackward0>)